In [18]:
import numpy as np
import pandas as pd

import os
import re
import h5py
import math
import shutil
import random
import nibabel
import tarfile
import nibabel as nib
import SimpleITK
import matplotlib.pyplot as plt
import torchvision
import pathlib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm
from nilearn import plotting
from nilearn.plotting import plot_anat, plot_roi
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

from einops import rearrange
from medpy import metric

In [19]:
if torch.cuda.is_available():
    print("CUDA is available. GPU will be used for training.")
    device = torch.device("cuda")
else:
    print("CUDA is not available. Training will be on CPU.")
    device = torch.device("cpu")

CUDA is not available. Training will be on CPU.


In [20]:
path_train = '/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Training/'
path_test = '/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Testing/'

In [21]:
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()
        self.encoder = nn.ModuleList([
            self.conv_block(in_channels, 64),
            self.conv_block(64, 128),
            self.conv_block(128, 256),
            self.conv_block(256, 512),
            self.conv_block(512, 1024)
        ])
        
        self.decoder = nn.ModuleList([
            self.up_conv_block(1024, 512),
            self.up_conv_block(512, 256),
            self.up_conv_block(256, 128),
            self.up_conv_block(128, 64)
        ])
        
        self.final_layer = nn.Conv2d(64, out_channels, kernel_size=1)

    def conv_block(self, in_channels, out_channels):
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )
        return block

    def up_conv_block(self, in_channels, out_channels):
        block = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        return block

    def forward(self, x):
        encoders = []
        for encode in self.encoder:
            x = encode(x)
            encoders.append(x)
            x = nn.MaxPool2d(2)(x)

        for decode in self.decoder:
            x = decode(x)
            skip_connection = encoders.pop()
            x = torch.cat((x, skip_connection), dim=1)

        return self.final_layer(x)

In [29]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx].replace('.jpg', '.png'))
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform is not None:
            image = self.transform(image)
            mask = self.transform(mask)
        
        return image, mask

# Define the data transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

# Define datasets and data loaders
train_dataset = SegmentationDataset(
    image_dir='/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Training', 
    mask_dir='/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Training', 
    transform=transform
)
test_dataset = SegmentationDataset(
    image_dir='/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Testing', 
    mask_dir='/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Testing', 
    transform=transform
)

train_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=False)

In [30]:
# Initialize the U-Net model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet(in_channels=3, out_channels=1).to(device)

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.BCEWithLogitsLoss()

# Training loop
num_epochs = 20
best_accuracy = 0.0

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0

    for i, (images, masks) in enumerate(train_loader):
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_function(outputs, masks)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * images.size(0)

    train_loss /= len(train_dataset)

    # Evaluation
    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for i, (images, masks) in enumerate(test_loader):
            images, masks = images.to(device), masks.to(device)

            outputs = model(images)
            loss = loss_function(outputs, masks)
            test_loss += loss.item() * images.size(0)

    test_loss /= len(test_dataset)

    print(f'Epoch: {epoch + 1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')

    if test_loss < best_accuracy or epoch == 0:
        torch.save(model.state_dict(), 'best_checkpoint_unet.model')
        best_accuracy = test_loss

print('Training complete. Best test loss: ', best_accuracy)

UnidentifiedImageError: cannot identify image file '/Users/nasifsafwan/Downloads/ML/BrainTumorResearch/tumordata/Training/.DS_Store'